In [1]:
# import tensorflow as tf
import numpy as np
import torch
import random
import pandas as pd
import os, sys
import time
import datetime
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss
from IPython.display import display, HTML

# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name())

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A40


In [2]:
import os
attribute_pair = "african_european"
embeddings_path = "attribute_embeddings_"+attribute_pair
if not os.path.exists(embeddings_path):
    os.makedirs(embeddings_path)

# discriminator = torch.load("classification_model_"+attribute_pair+"_9.pth")
import bert_model_2_extract_attributes_after_training
# import importlib
# importlib.reload(bert_model_2_extract_attributes_after_training)
BertForMaskedLM = bert_model_2_extract_attributes_after_training.BertForMaskedLM

model = BertForMaskedLM.from_pretrained("bert-base-uncased", attribute_pair, max_token_count=70, threshold=8)
# model.load_state_dict(discriminator.state_dict())
# model = torch.nn.DataParallel(model, device_ids=None)
for param in model.parameters():
    # print(param)
    param.requires_grad = False
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly ini

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocabs = tokenizer.get_vocab()
vv = dict((v,k) for k,v in vocabs.items())

In [4]:
# (vv)

In [5]:
# with open('../../DIIN/data/enwiki-20230320-pages-articles3.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df2 = pd.DataFrame(lines)[0]
# attribute_df2.reset_index(drop=True)
# attribute_df2

In [6]:

# torch.save(train_dataset, '2_attribute_tokenized/train_attribute_attributes_tokens.pt')


In [7]:


import time
import datetime
import pickle
from transformers.optimization import AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32





from transformers.optimization import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
# epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
# total_steps = len(test_dataloader) * epochs





def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# import numpy as np


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


import random
from tqdm.notebook import tqdm

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
def train(model, test_dataloader):
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
#     t = tqdm(range(start_epoch,epochs))

#     for epoch_i in t:

   

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Extraction...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    all_embeddings = []
    all_gen_labels = []
    all_attribute_token_id = []
    all_logits = []
    # Evaluate data for one epoch
    jj=0

    count_dict = {}
    for i in tokenizer.vocab.values():
        count_dict[i] = 0

    
    
    for batch in test_dataloader:
        jj=jj+1
        if jj%100==0:
            print(jj)

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        # ster_labels = batch[4]



        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            embeddings, token_ids, gender, logits, count_dict = model(b_input_ids, 
                                 # token_type_ids=None,
                                 count_dict=count_dict,
                                 attention_mask=b_input_mask, 
                                 # labels=b_target_labels,
                                )

            # print(len(res))
            # res = embeddings.cpu().detach().numpy()
            # print(len(res), len(b_target_labels))
            for key, iii in enumerate(embeddings):
                all_embeddings.append(list(iii.cpu().detach().numpy()))
                # print(all_embeddings)
                all_attribute_token_id.append(token_ids[key].cpu().detach().numpy().item())
                # print(all_attribute_token_id)
                all_gen_labels.append(gender[key])
                # print(all_gen_labels)
                all_logits.append(logits[key].cpu().detach().numpy().item())

        # if jj<=20000:
            if jj%30000==0:
                # ccc = {vv[k]:v for k,v in count_dict.items()}
                # print(ccc)
                with open(embeddings_path+"/embeddings_"+str(jj)+".pkl", 'wb') as f:
                    pickle.dump(all_embeddings, f)
                with open(embeddings_path+"/token_ids_"+str(jj)+".pkl", 'wb') as f:
                    pickle.dump(all_attribute_token_id, f)
                with open(embeddings_path+"/gender_"+str(jj)+".pkl", 'wb') as f:
                    pickle.dump(all_gen_labels, f)
                with open(embeddings_path+"/logits_"+str(jj)+".pkl", 'wb') as f:
                    pickle.dump(all_logits, f)
    
                all_embeddings = []
                all_gen_labels = []
                all_attribute_token_id = []
                all_logits = []
    
    # # ccc = {vv[k]:v for k,v in count_dict.items()}
    # # print(ccc)
    # with open(embeddings_path+"/embeddings_"+str(jj)+".pkl", 'wb') as f:
    #     pickle.dump(all_embeddings, f)
    # with open(embeddings_path+"/token_ids_"+str(jj)+".pkl", 'wb') as f:
    #     pickle.dump(all_attribute_token_id, f)
    # with open(embeddings_path+"/gender_"+str(jj)+".pkl", 'wb') as f:
    #     pickle.dump(all_gen_labels, f)
    # with open(embeddings_path+"/logits_"+str(jj)+".pkl", 'wb') as f:
    #     pickle.dump(all_logits, f)
                


In [8]:
train_dataset = torch.load('1_train_dataset1.pt')

In [9]:


train_dataloader = DataLoader(
            train_dataset, # The validation samples.
            sampler = SequentialSampler(train_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
            )
    
train(model, train_dataloader)
    # time.sleep(8)


Running Extraction...
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100

In [10]:
# with open('../../DIIN/data/intrinsic_word_level_data/enwiki-20230320-pages-articles3.txt') as file:
#     # with open('../iterative_adversarial/CDA/0_model/final_files/data/MBCDA2_jigsaw.txt') as file:
# # with open('../DIIN/data/enwiki-20230320-pages-articles8.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df6 = pd.DataFrame(lines)[0]
# attribute_df6.reset_index(drop=True)
# text_list = list(attribute_df6[5000:8000])

# text_input_tokenized = tokenizer.batch_encode_plus(text_list, truncation=True, max_length=50, padding=True, return_tensors="pt")
# inputs = text_input_tokenized["input_ids"].to("cuda")
# attention_mask = text_input_tokenized["attention_mask"]
# # text_input_tokenized
# inputs2 = inputs.clone()


# from torch.utils.data import TensorDataset

# train_dataset = TensorDataset(inputs2, attention_mask)


In [11]:
# with open('../../DIIN/data/intrinsic_word_level_data/enwiki-20230320-pages-articles3.txt') as file:
#     # with open('../iterative_adversarial/CDA/0_model/final_files/data/MBCDA2_jigsaw.txt') as file:
# # with open('../DIIN/data/enwiki-20230320-pages-articles8.txt') as file:
#     lines = [line.rstrip() for line in file]
# attribute_df6 = pd.DataFrame(lines)[0]
# attribute_df6.reset_index(drop=True)
# text_list = attribute_df6[:]

# text_list = list(text_list)

# ratio = 300000
# start_ind = 0
# end_ind = 0

# for i in range(int(len(text_list)/ratio)):
#     print(i)
#     end_ind+=ratio
#     text_input_tokenized = tokenizer.batch_encode_plus(text_list[start_ind:end_ind], truncation=True, max_length=150, padding=True, return_tensors="pt")
#     input_ids = text_input_tokenized["input_ids"].to("cuda")
#     attention_mask = text_input_tokenized["attention_mask"]
#     start_ind += ratio
    
#     from torch.utils.data import TensorDataset
    
#     train_dataset = TensorDataset(input_ids, attention_mask)
    




#     train_dataloader = DataLoader(
#             train_dataset, # The validation samples.
#             sampler = SequentialSampler(train_dataset), # Pull out batches sequentially.
#             batch_size = batch_size # Evaluate with this batch size.
#             )
    
#     # Start
#     start_epoch = 0
    
#     # Logger
#     losslogger = pd.DataFrame()
    
#     # Checkpoint name
#     checkpoint_name = 'checkpoint.pth.tar'
    
#     train(model, train_dataloader, i)
#     time.sleep(8)

In [12]:
len(text_list)

NameError: name 'text_list' is not defined

In [ ]:
1140679/300000

In [ ]:
# import os
# embeddings_path = "attribute_embeddings"
# if not os.path.exists(embeddings_path):
#     os.makedirs(embeddings_path)


# discriminator_outputs = discriminator(inputs)
# predictions = discriminator_outputs.logits
# preds = predictions.squeeze().tolist()
# # output_ids = list(map(identify_fake_tokens, inputs, preds))
# embeddings, token_ids, gender = tokenizer.batch_decode(inputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)

            
# with open("attribute_embeddings/embeddings.pkl", 'wb') as f:
#     pickle.dump(embeddings, f)
# with open("attribute_embeddings/token_ids.pkl", 'wb') as f:
#     pickle.dump(token_ids, f)
# with open("attribute_embeddings/gender.pkl", 'wb') as f:
#     pickle.dump(gender, f)


In [ ]:
# attention_mask = attention_mask.numpy()
# # attention_mask

In [ ]:
# vocabsss = tokenizer.get_vocab()
# vocabs = dict((v,k) for k,v in vocabsss.items())

In [ ]:
# for i in range(100):
#     index =i
#     text_in = [vocabs[i] for i in inputs2[index].cpu().numpy()]
#     # print(text_in)
#     try:
#         preddd = [i for i in preds[index]]
#     except:
#         preddd = preds
#     text_in = [vocabs[j] for k,j in enumerate(inputs2[index].cpu().numpy()) if attention_mask[index][k]==1]
#     preddd = [j for k,j in enumerate(preddd) if attention_mask[index][k]==1]
#     preddd = [j for k,j in enumerate(preddd) if (text_in[k].startswith('##')==False and text_in[k]!='.' and text_in[k]!='?'
#     and text_in[k]!='!' and text_in[k]!="," and text_in[k]!="n't"
#     and text_in[k]!="'m" and text_in[k]!="'s" and text_in[k]!="'ve" and text_in[k]!="'re"
#     and text_in[k]!="[UNK]")]# and text_in[k].startswith("(")==False and text_in[k].startswith(")")==False and text_in[k].startswith("-")==False)]
#     # and text_in[k].startswith(":")==False and text_in[k].startswith("(")==False and text_in[k].startswith(")")==False and text_in[k].startswith("-")==False)]
#     preddd = preddd[1:-1]
#     text_in = text_in[1:-1]
    
    
#     words = output_texts[index].split()
#     # print(text_in)
#     # print(words)
#     if len(words) == len(preddd):
#         s = colorize(words, NormalizeData(preddd))
#         # to display in ipython notebook
#         print(len(words), len(preddd))
#         print(NormalizeData(preddd))
#         display(HTML(s))
#     else:
#         print("process failed")

In [ ]:
# discriminator_outputs

In [ ]:
# preds

In [ ]:
# preddd

In [ ]:
# for i in tokenizer.vocab.values():
#     print(i)